# 1. 기본변수선정 : 가설검정을 통한 재무 데이터와 비재무 데이터의 선정

베이스라인에 사용한 변수는 56개로 같은 것을 나타내는 변수가 존재하고 있을 것이다. 이는 다중공선성 문제와 직결되기 때문에 가설검정을 통해 변수를 추려내려한다.
1. 재무 데이터(연속형): 휴폐업인 기업과 영업중인 기업의 재무 데이터 중 분포가 차이가 나는 것이 추후 분류 모델에 적용 시 도움이 될 것이므로 Two Sample T-Test를 진행한다.
2. 비재무 데이터(명목형) : 타겟값인 휴폐업 여부와 비재무 데이터 중 독립이 아닌 것 즉, 연관이 있는 것들이 추후 분류 모델에 적용 시 도움이 될 것이므로 Two Sample Chi-Squared-Test를 진행한다.

## 1-1. 휴폐업인 기업과 영업중인 기업간의 차이를 확인하기 위한 Two-Sample T-Test 진행


### - 귀무가설: 휴폐업인 기업과 영업중인 기업의 A 재무 데이터는 동일하다.
### - 대립가설: 휴폐업인 기업과 영업중인 기업의 A 재무 데이터는 달라서 두 집단을 구분할 수 있다.

### 1-1-1. 데이터 불러오기

In [1]:
import pandas as pd
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./custom_data/base_data.csv', encoding='cp949')
indurstry = pd.read_csv('./custom_data/feature_engineering_idurstry_code.csv', encoding='cp949')

In [3]:
# 베이스 라인에서 사용한 데이터와 업종중분류, 업종대분류 데이터 합치기
df_merge_indurstry = pd.merge(df, indurstry[['사업자등록번호', '업종중분류', '업종대분류']], on='사업자등록번호', how='left')

### 1-1-2. 데이터 정제(베이스라인과 동일)

In [4]:
def check_biz_no(df):
    """
    사업자등록번호가 여러 개인 기업들을 찾고, 각 경우에 따라 처리하는 함수입니다.
    ----------------------------------------
    input = 데이터프레임
    return = 정제한 데이터프레임
    ----------------------------------------
    """
    df_origin = df.copy()

    group_data = df.groupby(['기업명', '사업자등록번호'], as_index=False).size()

    duplicated_cor_name = group_data[group_data.duplicated(subset=['기업명'])]['기업명'].unique()

    result_dict = [{"case1_사업자등록번호":[]}, {"case2_사업자등록번호":[]}]

    # case1, case2에 해당하는 사업자등록번호 찾기
    for dup_name in duplicated_cor_name:
        dup_count = group_data.loc[group_data['기업명'] == dup_name]['size'].values.tolist()
        if len(set(dup_count)) == 1:
            for i in range(len(dup_count)-1):
                result_dict[0]["case1_사업자등록번호"].append(group_data.loc[group_data['기업명'] == dup_name].iloc[i]['사업자등록번호'])
        elif len(set(dup_count)) != 1:
            for i in range(1, len(dup_count)):
                result_dict[1]["case2_사업자등록번호"].append(group_data.loc[group_data['기업명'] == dup_name].iloc[i]['사업자등록번호'])
    
    # case1에 해당하는 경우 제거하기
    for case1_num in result_dict[0]["case1_사업자등록번호"]:
        idx = df[df['사업자등록번호']==case1_num].index
        df.drop(idx, inplace=True)

    # case2에 해당하는 경우 제거하기
    for case2_num in result_dict[1]["case2_사업자등록번호"]:
        idx = df[df['사업자등록번호']==case2_num].index
        df.drop(idx, inplace=True)
    
    print(f"사업자등록번호가 여러 개인 기업들을 기준에 따라 정제한 결과 {df_origin.shape[0] - df.shape[0]}개의 데이터가 제거되었습니다.")
    print(f"사업자등록번호를 기준으로 정제 후 데이터의 크기: {df.shape}")

In [5]:
check_biz_no(df_merge_indurstry)

사업자등록번호가 여러 개인 기업들을 기준에 따라 정제한 결과 6197개의 데이터가 제거되었습니다.
사업자등록번호를 기준으로 정제 후 데이터의 크기: (102945, 79)


In [6]:
df_finance_missing_value = df_merge_indurstry.copy()

In [7]:
finance_columns = df_finance_missing_value.columns[:46]

# 결측치를 0으로 변환
df_finance_missing_value[finance_columns] = df_finance_missing_value[finance_columns].fillna(0)

for column in finance_columns:
    df_finance_missing_value[column] = df_finance_missing_value[column].apply(lambda x: 0 if x == 1000000000000 else x)
    df_finance_missing_value[column] = df_finance_missing_value[column].apply(lambda x: 0 if x == 888888888889 else x)

In [8]:
print(f"정제 후 결측치의 개수: {df_finance_missing_value[finance_columns].isna().sum().sum()}개")
total = 0
for column in finance_columns:
    total += len(df_finance_missing_value.loc[(df_finance_missing_value[column] == 1000000000000) | (df_finance_missing_value[column] == 888888888889)])
print(f"정제 후 이상치의 개수: {total}개")

정제 후 결측치의 개수: 0개
정제 후 이상치의 개수: 0개


In [9]:
# 5개기업이 (-) 자본금을 가짐
count_minus_capital_stock = df_finance_missing_value.loc[df_finance_missing_value['자본금'] < 0].shape[0]
print(f"자본금이 음수인 기업의 개수: {count_minus_capital_stock}개")

자본금이 음수인 기업의 개수: 5개


In [10]:
# 0으로 바꿈
df_finance_missing_value['자본금'] = df_finance_missing_value['자본금'].apply(lambda x: 0 if x < 0 else x)
count_minus_capital_stock = df_finance_missing_value.loc[df_finance_missing_value['자본금'] < 0].shape[0]
print(f"정제 후 자본금이 음수인 기업의 개수: {count_minus_capital_stock}개")

정제 후 자본금이 음수인 기업의 개수: 0개


In [11]:
df_finance_missing_value.loc[df_finance_missing_value['자산총계'] == 0].iloc[:5, :46]

,사업자등록번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,...,비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
280,1018164892,20191231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
281,1018164892,20201231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282,1018164892,20211231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336,1018179639,20191231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337,1018179639,20201231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
count_zero_total_assets = df_finance_missing_value.loc[df_finance_missing_value['자산총계'] == 0].shape[0]
df_finance_missing_value = df_finance_missing_value[df_finance_missing_value['자산총계'] != 0]

print(f"자산의 이상치 제거 결과 {count_zero_total_assets}개의 데이터가 제거되었습니다.")

자산의 이상치 제거 결과 983개의 데이터가 제거되었습니다.


In [13]:
# 2018, 2022년도의 데이터 제거
df_finance_missing_value['결산년월'] = df_finance_missing_value['결산년월'].astype(str)
df_cleaning = df_finance_missing_value.loc[~df_finance_missing_value['결산년월'].str.contains('2018|2022')].reset_index(drop=True)

In [14]:
df_cleaning.shape

(72968, 79)

In [15]:
df_cleaning['휴폐업구분'].value_counts()

폐업      3775
폐업취소     541
휴업취소     314
휴업         1
Name: 휴폐업구분, dtype: int64

In [16]:
def change_target(value):
    """
    타겟값을 변환하기 위한 함수입니다.
    --------------------------------
    input = 휴폐업구분
    return = 폐업 or 휴업
    --------------------------------
    """
    if value in ['폐업', '휴업']:
        return 1
    return 0

In [17]:
df_cleaning['휴폐업구분'] = df_cleaning['휴폐업구분'].apply(change_target)

In [18]:
df_cleaning['휴폐업구분'].value_counts()

0    69192
1     3776
Name: 휴폐업구분, dtype: int64

### 1-1-3. Two Sample T-Test 계산

In [19]:
def calculate_ttest(df, column_list):
    """
    Two Sample T-Test를 수행하고 유의미한 변수를 리스트에 담아 반환하는 함수입니다.
    --------------------------------------------------------------------------
    input = 데이터프레임, t-test를 수행할 변수명(리스트)
    return = 유의미한 변수(리스트)
    --------------------------------------------------------------------------
    """
    df_active = df.loc[df['휴폐업구분'] == 0]
    df_closed = df.loc[df['휴폐업구분'] == 1]

    use_columns = []

    for column in column_list:
        closed = df_active[column]
        active = df_closed[column]
        _, p_value = stats.ttest_ind(active, closed)
        if p_value < 0.05:
            print(f"영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")
            use_columns.append(column)
    
    return use_columns

In [20]:
basic_finance_use_columns = calculate_ttest(df_cleaning, df_cleaning.columns[2:46])

영업중인 기업과 폐업한 기업의 유동자산 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 매출채권 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 비유동자산 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 유형자산 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 자산총계 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 유동부채 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 비유동부채 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 부  채  총  계 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 이익잉여금(결손금） 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 자본총계 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 매출액 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 판매비와관리비 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 영업이익（손실） 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 법인세비용차감전순손익 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 법인세비용 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 당기순이익(손실) 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 기업순이익률(%) 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 유보액/총자산(%) 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 금융비용대총비용비율(% 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 

### 1-1-4. 재무 데이터 가설 검정 결과
- 휴폐업인 기업과 영업 중인 기업 두 집단의 44개 재무 데이터에 대해 Two Sample T-Test를 진행한 결과 p-value가 0.05보다 작은 작은 24개의 변수 발견, <br>
  따라서 귀무가설을 기각하고 대립가설 채택.
- 선정된 재무 데이터 목록<br>
: 유동자산, 매출채권, 비유동자산, 유형자산, 자산총계, 유동부채, 비유동부채, 부채총계, 이익잉여금, 자본총계, 매출액, 판매비와관리비, 영업이익, 법인세비용차감전순손익, 법인세비용, 당기순이익, 기업순이익률, 유보액대총자산비율, 금융비용대총비용비율, 차입금의존도, 자기자본비율, 총자본회전율, 매출원가, 재고자산

## 1-2. 비재무 데이터와 휴폐업 여부의 연관성을 확인하기 위해 Two Sample Chi-Squared-Test 진행 

### - 귀무가설: 비재무 데이터 A와 휴폐업 여부는 독립이므로 연관이 없다.
### - 대립가설: 비재무 데이터 A와 휴폐업 여부는 독립이 아니므로 연관이 있다. 따라서 휴폐업 예측에 영향이 있을 것이다.

### 1-2-1. 비재무 데이터 불러오기

In [21]:
non_finance_columns = ['공기업구분 1.공기업 2.일반기업', '본점지점구분 1.본점 2.지점', '국외투자법인여부', 
                       '벤처기업여부', '공공기관유형', '업종중분류', '업종대분류', '업종']

### 1-2-2. Two Sample Chi-Squared-Test 계산

In [22]:
def calculate_chi_test(df, column_list):
    """
    Two Sample Chi Squared Test를 수행하고 유의미한 변수를 리스트에 담아 반환하는 함수입니다.
    --------------------------------------------------------------------------
    input = 데이터프레임, chi-squared-test를 수행할 변수명(리스트)
    return = 유의미한 변수(리스트)
    --------------------------------------------------------------------------
    """
    use_columns = []
    
    for colum in column_list:
        non_finance_data = df[colum]
        active_closed_data = df['휴폐업구분']

        data = pd.crosstab(non_finance_data, active_closed_data)
        _, pvalue, _, _ = stats.chi2_contingency(data, correction=False)
        if pvalue < 0.05:
            print(f"휴폐업과 {colum} 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.")
            use_columns.append(colum)
    
    return use_columns

In [23]:
basic_non_finance_use_columns = calculate_chi_test(df_cleaning, non_finance_columns)

휴폐업과 본점지점구분 1.본점 2.지점 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.
휴폐업과 국외투자법인여부 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.
휴폐업과 업종중분류 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.
휴폐업과 업종대분류 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.


### 1-2-3. 비재무 데이터 가설검정 결과

- 휴폐업 유무와 비재무 데이터 7개의 연관성을 확인하기 위해 Two Sample Chi-Squared-Test를 진행한 결과 p-value가 0.05보다 작은 변수 4개 발견, <br>
  따라서 귀무가설을 기각하고 대립가설 채택.
- 선정된 비재무 데이터 목록 <br>
: 본점지점구분, 국외투자법인여부, 업종중분류, 업종대분류

## 1-3. 검정 결과 유의미성 검증

- 통계적으로 재무 데이터와 비재무 데이터에서 유의미해 보이는 피처들을 찾았다.
- 따라서 베이스라인에서 사용한 데이터와 비교했을 때, 성능이 향상되면 검정 결과가 유의미하다는 것으로 간주하려한다.

In [24]:
# 검정 결과로 추려낸 재무 데이터 변수 목록
print(basic_finance_use_columns)

['유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채', '비유동부채', '부  채  총  계', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비', '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)', '유보액/총자산(%)', '금융비용대총비용비율(%', '차입금의존도(%)', '자기자본비율(%)', '총자본회전율(회)', '매출원가', '재고자산']


In [25]:
basic_use_columns = basic_finance_use_columns + basic_non_finance_use_columns + ['휴폐업구분']

In [26]:
# 검정 결과에 따른 학습 데이터 생성
df_basic_ttest = df_cleaning[basic_use_columns]

In [27]:
# 베이스 라인에서 사용한 학습 데이터 생성
df_baseline = df_cleaning.drop(['기업접두명', '기업접미명', '기업영문명', '기업규모', '개인법인구분 1.법인 2.개인', 
                                '상장코드', '산업코드차수', '산업코드1', '산업코드2', '산업코드3', '본점기업코드', '설립일자', 
                                '설립구분', '상장일자', '주요사업내용', '국가명', '홈페이지URL', '대표자명',
                                '직원수', '종료일자', '시작일자', '상태발생일자', '중견기업보호여부'], axis=1)

In [28]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score

In [29]:
def preprocess_and_split_data(X, y):
    """
    전처리와 데이터 분할을 수행하는 함수입니다.
    """
    X = pd.get_dummies(X)
    X = X.rename(columns = lambda x:x.replace(',', ' '))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print(f"훈련 데이터셋의 입력 데이터의 크기: {X_train.shape}, 타겟값의 크기: {y_train.shape}")
    print(f"테스트 데이터셋의 입력 데이터의 크기: {X_test.shape}, 타겟값의 크기: {y_test.shape}")

    return X_train, y_train, X_test, y_test

def get_clf_eval(y_test, pred=None):
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

def train_model(X_train, y_train, X_test, y_test):
    model = {
        'LGBM':LGBMClassifier(),
        'XGB':XGBClassifier(),
        'RandomForest':RandomForestClassifier(),
        'LogisticRegression':LogisticRegression(),
        'NaiveBayes':GaussianNB()
    }
    for algorithm in model.keys():
        estimator = model[algorithm]
        if algorithm == 'XGB':
            estimator.fit(X_train, y_train, eval_metric='logloss')
        else:
            estimator.fit(X_train, y_train)
        preds = estimator.predict(X_test)
        print(f"{algorithm}모델 ", end='')
        get_clf_eval(y_test, preds)

In [30]:
X_basic = df_baseline.drop(['사업자등록번호', '결산년월', '업종중분류', '업종대분류', '기업명', '휴폐업구분'], axis=1)
y_basic = df_baseline['휴폐업구분']

X_stats = df_basic_ttest.drop(['업종대분류', '휴폐업구분'], axis=1)
y_stats = df_basic_ttest['휴폐업구분']

In [34]:
print(f"베이스라인에 사용된 피처\n {X_basic.columns}")
print(f"통계검정 재무/비재무에 사용된 피처\n {X_stats.columns}")

베이스라인에 사용된 피처
 Index(['유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채', '비유동부채', '부  채  총  계',
       '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비', '영업이익（손실）',
       '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)', '유보액/총자산(%)',
       '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)', '수지비율(%)',
       '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)', '금융비용대총비용비율(%',
       '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)', '유동부채비율(%)',
       '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)', '매출채권회전율(회)',
       '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산', '업종', '공기업구분 1.공기업 2.일반기업',
       '본점지점구분 1.본점 2.지점', '국외투자법인여부', '벤처기업여부', '공공기관유형'],
      dtype='object')
통계검정 재무/비재무에 사용된 피처
 Index(['유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채', '비유동부채', '부  채  총  계',
       '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비', '영업이익（손실）', '법인세비용차감전순손익',
       '법인세비용', '당기순이익(손실)', '기업순이익률(%)', '유보액/총자산(%)', '금융비용대총비용비율(%',
       '차입금의존도(%)', '자기자본비율(%)', '총자본회전율(회)', '매출원가', '재고자산'

In [35]:
print('=======================베이스 라인 데이터=============================')
# 베이스라인 데이터 
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_basic, y_basic)
train_model(X_train, y_train, X_test, y_test)

print('========================T-Test 기본 데이터===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats, y_stats)
train_model(X_train, y_train, X_test, y_test)

=======================베이스 라인 데이터=============================
훈련 데이터셋의 입력 데이터의 크기: (58374, 59), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 59), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9753, 정밀도: 0.8520, 재현율: 0.6331, F1: 0.7264
XGB모델 정확도: 0.9757, 정밀도: 0.8536, 재현율: 0.6411, F1: 0.7322
RandomForest모델 정확도: 0.9751, 정밀도: 0.8781, 재현율: 0.6013, F1: 0.7138
LogisticRegression모델 정확도: 0.9479, 정밀도: 0.2727, 재현율: 0.0040, F1: 0.0078
NaiveBayes모델 정확도: 0.1050, 정밀도: 0.0529, 재현율: 0.9642, F1: 0.1003
========================T-Test 기본 데이터===========================
훈련 데이터셋의 입력 데이터의 크기: (58374, 95), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 95), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9752, 정밀도: 0.8541, 재현율: 0.6278, F1: 0.7237
XGB모델 정확도: 0.9759, 정밀도: 0.8607, 재현율: 0.6384, F1: 0.7331
RandomForest모델 정확도: 0.9753, 정밀도: 0.8880, 재현율: 0.5987, F1: 0.7152
LogisticRegression모델 정확도: 0.9482, 정밀도: 0.4000, 재현율: 0.0026, F1: 0.0053
NaiveBayes모델 정확도: 0.9428, 정밀도: 0.0833, 재현율: 0.0106, F1: 0.0188


### 1-3-1. 정리
- 베이스라인 모델에 사용한 알고리즘인 LGBM을 사용하여 기존에 사용했던 데이터와 가설 검정으로 추려낸 데이터를 각각 학습시켜 비교해보았다.
- 두 집단의 유의미한 차이를 보인 변수들(=가설검정으로 추려낸 변수들)로 학습한 결과 정밀도, 재현율, F1-Score가 약간 상승하는 것을 확인했으므로 검정 결과가 유의미했음을 확인했다.

# 2. 재무비율 데이터 선정
현재 재무 비율 데이터는 210개로 매우 많은 상황이다. 따라서 이를 통계검정을 통해 유의미하다고 판단된 재무 비율만 사용하기로 한다.
- 재무 비율 데이터(연속형): 휴폐업인 기업과 영업중인 기업의 재무 비율 데이터 중 분포가 차이가 나는 것이 추후 분류 모델에 적용 시 도움이 될 것이므로 Two Sample T-Test를 진행한다.

## 2-1. 휴폐업인 기업과 영업중인 기업간의 제무 비율 데이터 차이를 확인하기 위한 Two-Sample T-Test 진행

### 2-1-1. 데이터 불러오기 및 전처리

In [36]:
import pandas as pd

finance_ratio = pd.read_csv('./custom_data/all_finance_data.csv', encoding='cp949')
df_active_closed = pd.read_csv('./custom_data/active_closed_all.csv', encoding='cp949')

In [37]:
# 통계검정을 위해 휴폐업구분 데이터 합치기
df_ratio = pd.merge(finance_ratio, df_active_closed[['사업자등록번호', '휴폐업구분']], on='사업자등록번호', how='left')

In [38]:
# merge를 위해 문자열로 변환
df_ratio['사업자등록번호'] = df_ratio['사업자등록번호'].astype(str)
df_ratio['결산년월'] = df_ratio['결산년월'].astype(str)

df_cleaning['사업자등록번호'] = df_cleaning['사업자등록번호'].astype(str)
df_cleaning['결산년월'] = df_cleaning['결산년월'].astype(str)

In [39]:
# 전처리한 데이터와 동일한 row를 사용하기 위해 합치기
df_cleaning_ratio = pd.merge(df_cleaning[['사업자등록번호', '결산년월']], df_ratio, on=['사업자등록번호', '결산년월'], how='left')

In [40]:
df_cleaning_ratio.shape

(72968, 213)

In [41]:
# 타겟값 인코딩
df_cleaning_ratio['휴폐업구분'] = df_cleaning_ratio['휴폐업구분'].apply(change_target)

In [42]:
df_cleaning_ratio['휴폐업구분'].value_counts()

0    69192
1     3776
Name: 휴폐업구분, dtype: int64

### 2-1-2. Two Sample T-Test 계산 (기본 재무 비율)

In [43]:
df_ratio_active = df_cleaning_ratio.loc[df_cleaning_ratio['휴폐업구분'] == 0]
df_ratio_closed = df_cleaning_ratio.loc[df_cleaning_ratio['휴폐업구분'] == 1]

In [44]:
# 기본 재무 비율 목록
df_cleaning_ratio.columns[2:32]

Index(['총자본증가율', '영업이익증가율', '당기순이익증가율', '자기자본증가율', '매출액증가율', '매출총이익률',
       '매출액영업이익률', '매출액경상이익률', '매출액순이익률', '총자산영업이익률', '자기자본영업이익률', '자기자본순이익률',
       '금융비용부담률', '수지비율', '사내유보대자기자본비율', '총자산순이익률', '총자본회전율', '자기자본회전율',
       '타인자본회전율', '유동자산회전율', '재고자산회전율', '당좌자산회전율', '순운전자본회전율', '운전자본회전율',
       '유보율', '자기자본비율', '유동비율', '당좌비율', '재고자산대순운전자본비율', '매출채권대매입채무비율'],
      dtype='object')

In [45]:
for column in df_cleaning_ratio.columns[2:32]:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율 데이터에서 영업중인 기업과 폐업한 기업의 총자본회전율 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율 데이터에서 영업중인 기업과 폐업한 기업의 자기자본비율 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율 데이터에서 영업중인 기업과 폐업한 기업의 유동비율 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-3. Two Sample T-Test 계산 (기본 재무 비율(차이))

In [46]:
# 기본 재무 비율(차이) 목록
df_cleaning_ratio.columns[32:62]

Index(['총자본증가율(차이)', '영업이익증가율(차이)', '당기순이익증가율(차이)', '자기자본증가율(차이)',
       '매출액증가율(차이)', '매출총이익률(차이)', '매출액영업이익률(차이)', '매출액경상이익률(차이)',
       '매출액순이익률(차이)', '총자산영업이익률(차이)', '자기자본영업이익률(차이)', '자기자본순이익률(차이)',
       '금융비용부담률(차이)', '수지비율(차이)', '사내유보대자기자본비율(차이)', '총자산순이익률(차이)',
       '총자본회전율(차이)', '자기자본회전율(차이)', '타인자본회전율(차이)', '유동자산회전율(차이)',
       '재고자산회전율(차이)', '당좌자산회전율(차이)', '순운전자본회전율(차이)', '운전자본회전율(차이)', '유보율(차이)',
       '자기자본비율(차이)', '유동비율(차이)', '당좌비율(차이)', '재고자산대순운전자본비율(차이)',
       '매출채권대매입채무비율(차이)'],
      dtype='object')

In [47]:
for column in df_cleaning_ratio.columns[32:62]:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 금융비용부담률(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 유동자산회전율(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 당좌자산회전율(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 자기자본비율(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 유동비율(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(차이) 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율(차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-3. Two Sample T-Test 계산 (기본 재무 비율(비율))

In [48]:
df_cleaning_ratio.columns[62:92]

Index(['총자본증가율(비율)', '영업이익증가율(비율)', '당기순이익증가율(비율)', '자기자본증가율(비율)',
       '매출액증가율(비율)', '매출총이익률(비율)', '매출액영업이익률(비율)', '매출액경상이익률(비율)',
       '매출액순이익률(비율)', '총자산영업이익률(비율)', '자기자본영업이익률(비율)', '자기자본순이익률(비율)',
       '금융비용부담률(비율)', '수지비율(비율)', '사내유보대자기자본비율(비율)', '총자산순이익률(비율)',
       '총자본회전율(비율)', '자기자본회전율(비율)', '타인자본회전율(비율)', '유동자산회전율(비율)',
       '재고자산회전율(비율)', '당좌자산회전율(비율)', '순운전자본회전율(비율)', '운전자본회전율(비율)', '유보율(비율)',
       '자기자본비율(비율)', '유동비율(비율)', '당좌비율(비율)', '재고자산대순운전자본비율(비율)',
       '매출채권대매입채무비율(비율)'],
      dtype='object')

In [49]:
for column in df_cleaning_ratio.columns[62:92]:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(비율) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(비율) 데이터에서 영업중인 기업과 폐업한 기업의 자기자본영업이익률(비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(비율) 데이터에서 영업중인 기업과 폐업한 기업의 자기자본비율(비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-4. Two Sample T-Test 계산 (기본 재무 비율(top10 차이))

In [50]:
top10_difference = df_cleaning_ratio.columns[df_cleaning_ratio.columns.str.contains('top10_차이')]

In [51]:
for column in top10_difference:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 금융비용부담률(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자본회전율(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 유동비율(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_차이) 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율(top10_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-5. Two Sample T-Test 계산 (기본 재무 비율(top10 비율))

In [52]:
top10_ratio = df_cleaning_ratio.columns[df_cleaning_ratio.columns.str.contains('top10_비율')]

In [53]:
for column in top10_ratio:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 금융비용부담률(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자본회전율(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 자기자본비율(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 유동비율(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(top10_비율) 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율(top10_비율) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-6. Two Sample T-Test 계산 (기본 재무 비율(all 차이))

In [54]:
all_difference = df_cleaning_ratio.columns[df_cleaning_ratio.columns.str.contains('all_차이')]

In [55]:
for column in all_difference:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 금융비용부담률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 총자본회전율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 유동비율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_차이) 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-7. Two Sample T-Test 계산 (기본 재무 비율(all 비율))

In [56]:
all_ratio = df_cleaning_ratio.columns[df_cleaning_ratio.columns.str.contains('all_비율')]

In [57]:
for column in all_difference:
    closed = df_ratio_active[column]
    active = df_ratio_closed[column]
    _, p_value = stats.ttest_ind(active, closed)
    if p_value < 0.05:
        print(f"재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 {column} 변수는 차이가 있다. 따라서 유의미한 변수이다.")

재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자산영업이익률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 금융비용부담률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자산순이익률(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 총자본회전율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 유동비율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.
재무비율(all_비율) 데이터에서 영업중인 기업과 폐업한 기업의 당좌비율(all_차이) 변수는 차이가 있다. 따라서 유의미한 변수이다.


### 2-1-8. 검정 결과
- 기본 재무 비율의 경우 총자산영업이익률, 총자산순이익률, 총자본회전율, 자기자본비율, 유동비율, 당좌비율이 유의미하다고 나왔다.
- 이에 대한 파생변수들(차이, 비율, top10_차이, ...) 중에서 기본 재무 비율과 가장 많이 겹치는 것으로 선정하려 한다.
- 따라서 top10_비율이 가장 많이 겹치므로 기본 재무 비율과 top10_비율을 사용해보려 한다. 

## 2-2. 검정 결과 유의미성 검증

### 2-2-1. top10_비율을 제외하고 기본 재무 비율만 사용해보기

In [58]:
use_ratio_columns = ['사업자등록번호', '결산년월', '총자산영업이익률', '총자산순이익률', '총자본회전율', '자기자본비율', '유동비율', '당좌비율']
df_final_ratio = df_cleaning_ratio[use_ratio_columns]

In [59]:
# 데이터를 합치기 위해 t-test에서 사용한 변수에 사업자등록번호, 결산년월 추가하기
for_merge_columns = basic_use_columns + ['사업자등록번호', '결산년월']

In [60]:
df_ttest = df_cleaning[for_merge_columns]

In [61]:
# merge를 위해 문자열로 변환
df_ttest['사업자등록번호'] = df_ttest['사업자등록번호'].astype(str)
df_ttest['결산년월'] = df_ttest['결산년월'].astype(str)

df_final_ratio['사업자등록번호'] = df_final_ratio['사업자등록번호'].astype(str)
df_final_ratio['결산년월'] = df_final_ratio['결산년월'].astype(str)

In [62]:
# 학습을 위해 데이터 합치기
df_basic_ratio = pd.merge(df_final_ratio, df_ttest, on=['사업자등록번호', '결산년월'], how='left')

In [64]:
X_ratio_stats = df_basic_ratio.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats = df_basic_ratio['휴폐업구분']

In [66]:
print(f"통계검정 재무비율에 사용된 피처\n {X_ratio_stats.columns}")

통계검정 재무비율에 사용된 피처
 Index(['총자산영업이익률', '총자산순이익률', '총자본회전율', '자기자본비율', '유동비율', '당좌비율', '유동자산',
       '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채', '비유동부채', '부  채  총  계',
       '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비', '영업이익（손실）', '법인세비용차감전순손익',
       '법인세비용', '당기순이익(손실)', '기업순이익률(%)', '유보액/총자산(%)', '금융비용대총비용비율(%',
       '차입금의존도(%)', '자기자본비율(%)', '총자본회전율(회)', '매출원가', '재고자산',
       '본점지점구분 1.본점 2.지점', '국외투자법인여부', '업종중분류'],
      dtype='object')


In [67]:
print('======================T-Test 기본+비율 데이터=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats, y_ratio_stats)
train_model(X_train, y_train, X_test, y_test)

======================T-Test 기본+비율 데이터=========================
훈련 데이터셋의 입력 데이터의 크기: (58374, 101), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 101), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9742, 정밀도: 0.8452, 재현율: 0.6146, F1: 0.7117
XGB모델 정확도: 0.9762, 정밀도: 0.8640, 재현율: 0.6397, F1: 0.7352
RandomForest모델 정확도: 0.9751, 정밀도: 0.8826, 재현율: 0.5974, F1: 0.7125
LogisticRegression모델 정확도: 0.9483, 정밀도: 0.5000, 재현율: 0.0040, F1: 0.0079
NaiveBayes모델 정확도: 0.9445, 정밀도: 0.1014, 재현율: 0.0093, F1: 0.0170


### 2-2-2. top10_비율을 포함해서 사용해보기

In [68]:
use_ratio__and_top10_columns = ['사업자등록번호', '결산년월', '총자산영업이익률', '총자산순이익률', '총자본회전율', '자기자본비율', '유동비율', '당좌비율',
                                '총자산영업이익률(top10_비율)', '총자산순이익률(top10_비율)', '총자본회전율(top10_비율)', '자기자본비율(top10_비율)', 
                                '유동비율(top10_비율)', '당좌비율(top10_비율)']
df_final_ratio_add_top10 = df_cleaning_ratio[use_ratio__and_top10_columns]

In [69]:
# merge를 위해 문자열로 변환
df_final_ratio_add_top10['사업자등록번호'] = df_final_ratio_add_top10['사업자등록번호'].astype(str)
df_final_ratio_add_top10['결산년월'] = df_final_ratio_add_top10['결산년월'].astype(str)

In [70]:
# 학습을 위해 데이터 합치기
df_basic_ratio_and_top10 = pd.merge(df_final_ratio_add_top10, df_ttest, on=['사업자등록번호', '결산년월'], how='left')

In [74]:
X_ratio_and_top10_stats = df_basic_ratio_and_top10.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats = df_basic_ratio_and_top10['휴폐업구분']

In [75]:
print(f"통계검정 재무비율/파생재무비율에 사용된 피처\n {X_ratio_and_top10_stats.columns}")

통계검정 재무비율/파생재무비율에 사용된 피처
 Index(['총자산영업이익률', '총자산순이익률', '총자본회전율', '자기자본비율', '유동비율', '당좌비율',
       '총자산영업이익률(top10_비율)', '총자산순이익률(top10_비율)', '총자본회전율(top10_비율)',
       '자기자본비율(top10_비율)', '유동비율(top10_비율)', '당좌비율(top10_비율)', '유동자산', '매출채권',
       '비유동자산', '유형자산', '자산총계', '유동부채', '비유동부채', '부  채  총  계', '이익잉여금(결손금）',
       '자본총계', '매출액', '판매비와관리비', '영업이익（손실）', '법인세비용차감전순손익', '법인세비용',
       '당기순이익(손실)', '기업순이익률(%)', '유보액/총자산(%)', '금융비용대총비용비율(%', '차입금의존도(%)',
       '자기자본비율(%)', '총자본회전율(회)', '매출원가', '재고자산', '본점지점구분 1.본점 2.지점',
       '국외투자법인여부', '업종중분류'],
      dtype='object')


In [76]:
print('==================T-Test 기본+비율+top10비율 데이터====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats, y_ratio_and_top10_stats)
train_model(X_train, y_train, X_test, y_test)

==================T-Test 기본+비율+top10비율 데이터====================
훈련 데이터셋의 입력 데이터의 크기: (58374, 107), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 107), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9755, 정밀도: 0.8590, 재현율: 0.6291, F1: 0.7263
XGB모델 정확도: 0.9764, 정밀도: 0.8637, 재현율: 0.6464, F1: 0.7394
RandomForest모델 정확도: 0.9752, 정밀도: 0.8861, 재현율: 0.5974, F1: 0.7136
LogisticRegression모델 정확도: 0.9483, 정밀도: 0.5000, 재현율: 0.0040, F1: 0.0079
NaiveBayes모델 정확도: 0.9452, 정밀도: 0.0909, 재현율: 0.0066, F1: 0.0123


### 2-2-3. 정리
- 재무 비율 데이터의 경우 단독으로 학습에 사용하면 악영향을 끼친다.
- 재무 데이터와 비슷한 상관성을 나타내기 때문이라 추정하여 단독으로 사용하지 않고 이들을 하나의 지표로 나타내어 사용하도록 한다.
- 재무비율 중 값이 클수록 기업에 긍정적인 영향을 미치는 요소들을 뽑아 만든 KSG 지표를 사용한다.

# 3. KSG 지표

In [77]:
ksg = pd.read_csv('./KSG_date_number.csv', index_col=0)

In [78]:
ksg['사업자등록번호'] = ksg['사업자등록번호'].astype(str)
ksg['결산년월'] = ksg['결산년월'].astype(str)

In [79]:
df_basic_ttest_ksg = pd.merge(df_ttest, ksg[['사업자등록번호', '결산년월', 'count']], on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg = pd.merge(df_basic_ratio, ksg[['사업자등록번호', '결산년월', 'count']], on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg = pd.merge(df_basic_ratio_and_top10, ksg[['사업자등록번호', '결산년월', 'count']], on=['사업자등록번호', '결산년월'], how='left')

In [80]:
ksg_column = calculate_chi_test(df_ratio_top10_ksg, ['count'])

휴폐업과 count 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.


In [81]:
X_stats_ksg = df_basic_ttest_ksg.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_ksg = df_basic_ttest_ksg['휴폐업구분']

X_ratio_stats_ksg = df_ratio_ksg.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_ksg = df_ratio_ksg['휴폐업구분']

X_ratio_and_top10_stats_ksg = df_ratio_top10_ksg.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg = df_ratio_top10_ksg['휴폐업구분']

In [82]:
print(f"통계검정 재무/비재무/재무등급에 사용된 피처\n {X_stats_ksg}")
print(f"통계검정 재무/비재무/재무비율/재무등급에 사용된 피처\n {X_ratio_stats_ksg}")
print(f"통계검정 재무/비재무/재무비율/파생재무비율/재무등급에 사용된 피처\n {X_ratio_and_top10_stats_ksg}")

통계검정 재무/비재무/재무등급에 사용된 피처
             유동자산       매출채권        비유동자산         유형자산         자산총계  \
0      2168837.0    94085.0   34151633.0    1209369.0   36320470.0   
1      2272260.0    78612.0   38329680.0    1091153.0   40601940.0   
2      1175030.0   273522.0   37135811.0     993025.0   38310842.0   
3       449696.0    18495.0   20926406.0   20909156.0   21376102.0   
4       509847.0     1044.0   21038587.0   21033337.0   21548434.0   
...          ...        ...          ...          ...          ...   
72963  8501297.0  3412249.0    6644464.0    5839959.0   15145761.0   
72964  9192992.0  2918329.0    9735232.0    9409374.0   18928223.0   
72965   475283.0        0.0  309796969.0  309796969.0  310272251.0   
72966  1474607.0        0.0  366181956.0  366181956.0  367656563.0   
72967  2540577.0  1201346.0  422469161.0  420110684.0  425009738.0   

              유동부채        비유동부채   부  채  총  계  이익잉여금(결손금）        자본총계  ...  \
0       10553878.0    5240445.0   15794323.0   3708092.0

In [83]:
print('=======================T-Test 기본 데이터+KSG==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg, y_stats_ksg)
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg, y_ratio_stats_ksg)
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg, y_ratio_and_top10_stats_ksg)
train_model(X_train, y_train, X_test, y_test)

=======================T-Test 기본 데이터+KSG==========================
훈련 데이터셋의 입력 데이터의 크기: (58374, 106), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 106), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9748, 정밀도: 0.8512, 재현율: 0.6212, F1: 0.7182
XGB모델 정확도: 0.9751, 정밀도: 0.8535, 재현율: 0.6252, F1: 0.7217
RandomForest모델 정확도: 0.9752, 정밀도: 0.8907, 재현율: 0.5934, F1: 0.7122
LogisticRegression모델 정확도: 0.9483, 정밀도: 0.5000, 재현율: 0.0040, F1: 0.0079
NaiveBayes모델 정확도: 0.9428, 정밀도: 0.0833, 재현율: 0.0106, F1: 0.0188
=====================T-Test 기본+비율 데이터+KSG========================
훈련 데이터셋의 입력 데이터의 크기: (58374, 112), 타겟값의 크기: (58374,)
테스트 데이터셋의 입력 데이터의 크기: (14594, 112), 타겟값의 크기: (14594,)
LGBM모델 정확도: 0.9755, 정밀도: 0.8538, 재현율: 0.6344, F1: 0.7280
XGB모델 정확도: 0.9755, 정밀도: 0.8590, 재현율: 0.6291, F1: 0.7263
RandomForest모델 정확도: 0.9747, 정밀도: 0.8799, 재현율: 0.5921, F1: 0.7078
LogisticRegression모델 정확도: 0.9483, 정밀도: 0.5000, 재현율: 0.0040, F1: 0.0079
NaiveBayes모델 정확도: 0.9445, 정밀도: 0.1014, 재현율: 0.0093, F1: 0.0170
=================T-Test 기본+비율

# 3. 통계청 자료

In [84]:
address = pd.read_csv('./custom_data/address.csv', index_col=0)

In [85]:
address['사업자등록번호'] = address['사업자등록번호'].astype(str)

In [86]:
df_ttest_address = pd.merge(df_ttest, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')
df_ratio_address = pd.merge(df_basic_ratio, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')
df_ratio_top10_address = pd.merge(df_basic_ratio_and_top10, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')

df_ttest_ksg_address = pd.merge(df_basic_ttest_ksg, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')
df_ratio_ksg_address = pd.merge(df_ratio_ksg, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')
df_ratio_top10_ksg_address = pd.merge(df_ratio_top10_ksg, address[['사업자등록번호', '시도']], on='사업자등록번호', how='left')

In [87]:
# 일부 주소가 없는 기업들(검색해도 나오지 않는 기업들) 제거
df_ttest_address.dropna(inplace=True)
df_ratio_address.dropna(inplace=True)
df_ratio_top10_address.dropna(inplace=True)

df_ttest_ksg_address.dropna(inplace=True)
df_ratio_ksg_address.dropna(inplace=True)
df_ratio_top10_ksg_address.dropna(inplace=True)

In [88]:
# 인덱스 번호 초기화
df_ttest_address = df_ttest_address.reset_index(drop=True)
df_ratio_address = df_ratio_address.reset_index(drop=True)
df_ratio_top10_address = df_ratio_top10_address.reset_index(drop=True)

df_ttest_ksg_address = df_ttest_ksg_address.reset_index(drop=True)
df_ratio_ksg_address = df_ratio_ksg_address.reset_index(drop=True)
df_ratio_top10_ksg_address = df_ratio_top10_ksg_address.reset_index(drop=True)

In [89]:
statistics_2019 = pd.read_csv('./region_peup_19.csv')
statistics_2020 = pd.read_csv('./region_peup_20.csv')
statistics_2021 = pd.read_csv('./region_peup_21.csv')

In [90]:
statistics_2019['년도'] = '2019'
statistics_2020['년도'] = '2020'
statistics_2021['년도'] = '2021'

In [91]:
# concat을 위해 년도 데이터 생성
df_ttest_address['년도'] = df_ttest_address['결산년월'].str[:4]
df_ratio_address['년도'] = df_ratio_address['결산년월'].str[:4]
df_ratio_top10_address['년도'] = df_ratio_top10_address['결산년월'].str[:4]

df_ttest_ksg_address['년도'] = df_ttest_ksg_address['결산년월'].str[:4]
df_ratio_ksg_address['년도'] = df_ratio_ksg_address['결산년월'].str[:4]
df_ratio_top10_ksg_address['년도'] = df_ratio_top10_ksg_address['결산년월'].str[:4]

In [93]:
# 지방지표 자료와 합치기 전 데이터의 형태 확인하기
print(df_ttest_address.shape)
print(df_ratio_address.shape)
print(df_ratio_top10_address.shape)
print(df_ttest_ksg_address.shape)
print(df_ratio_ksg_address.shape)
print(df_ratio_top10_ksg_address.shape)

(72956, 33)
(72956, 39)
(72956, 45)
(72956, 34)
(72956, 40)
(72956, 46)


In [94]:
# 2019~2021년의 지방지표 자료 합치기
statistics_total = pd.concat([statistics_2019, statistics_2020, statistics_2021]).reset_index(drop=True)

In [95]:
# merge를 위한 컬럼명 변경
statistics_total.columns = ['시도', '기업수', '폐업수', '폐업률', '혼자사는사람', '취업자증감', '고용률', '경제활동참가률', '실업률',
                            '광공업생산지수', '재정자립도', '특허', '가구별소득', '노인비율', '외국인비율', '인구수', '도시면적',
                            '전기사용량', '년도']

In [96]:
df_ttest_statistics = pd.merge(df_ttest_address, statistics_total, on=['시도', '년도'], how='left')
df_ratio_statistics = pd.merge(df_ratio_address, statistics_total, on=['시도', '년도'], how='left')
df_ratio_top10_statistics = pd.merge(df_ratio_top10_address, statistics_total, on=['시도', '년도'], how='left')

df_ttest_ksg_statistics = pd.merge(df_ttest_ksg_address, statistics_total, on=['시도', '년도'], how='left')
df_ratio_ksg_statistics = pd.merge(df_ratio_ksg_address, statistics_total, on=['시도', '년도'], how='left')
df_ratio_top10_ksg_statistics = pd.merge(df_ratio_top10_ksg_address, statistics_total, on=['시도', '년도'], how='left')

In [97]:
df_ttest_statistics.head()

,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,이익잉여금(결손금）,자본총계,...,실업률,광공업생산지수,재정자립도,특허,가구별소득,노인비율,외국인비율,인구수,도시면적,전기사용량
0,2168837.0,94085.0,34151633.0,1209369.0,36320470.0,10553878.0,5240445.0,15794323.0,3708092.0,20526147.0,...,4.4,97.8,76.5,52270.0,6595.0,15.2,28.97,9639541.0,605512630.0,47167206.0
1,2272260.0,78612.0,38329680.0,1091153.0,40601940.0,863144.0,15496117.0,16359261.0,4230449.0,24242679.0,...,4.6,84.7,76.1,53124.0,6575.0,16.1,25.09,9586195.0,605680193.0,45787926.0
2,1175030.0,273522.0,37135811.0,993025.0,38310842.0,9494272.0,4901383.0,14395654.0,5169522.0,23915187.0,...,4.8,89.3,75.6,54042.0,6826.0,16.8,23.83,9472127.0,605680264.0,47295807.0
3,449696.0,18495.0,20926406.0,20909156.0,21376102.0,3304660.0,11357238.0,14661898.0,-7398595.0,6714204.0,...,4.4,97.8,76.5,52270.0,6595.0,15.2,28.97,9639541.0,605512630.0,47167206.0
4,509847.0,1044.0,21038587.0,21033337.0,21548434.0,2847579.0,12704550.0,15552129.0,-8116494.0,5996305.0,...,4.6,84.7,76.1,53124.0,6575.0,16.1,25.09,9586195.0,605680193.0,45787926.0


In [103]:
# Two Sample T-Test를 진행할 통계청 데이터 선정
ttest_columns = ['혼자사는사람', '취업자증감', '고용률', '경제활동참가률', '실업률', '광공업생산지수',
                '재정자립도', '특허', '가구별소득', '노인비율', '외국인비율', '인구수', '도시면적', '전기사용량']

In [102]:
use_statistics_columns = calculate_ttest(df_ttest_statistics, ttest_columns)

영업중인 기업과 폐업한 기업의 혼자사는사람 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 고용률 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 경제활동참가률 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 광공업생산지수 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 재정자립도 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 가구별소득 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 노인비율 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 외국인비율 변수는 차이가 있다. 따라서 유의미한 변수이다.
영업중인 기업과 폐업한 기업의 도시면적 변수는 차이가 있다. 따라서 유의미한 변수이다.
휴폐업과 시도 데이터는 연관성이 나타난다. 따라서 유의미한 변수이다.


In [107]:
X_stats_statistics = df_ttest_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_statistics = df_ttest_statistics['휴폐업구분']

X_ratio_stats_statistics = df_ratio_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_statistics = df_ratio_statistics['휴폐업구분']

X_ratio_and_top10_stats_statistics = df_ratio_top10_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_statistics = df_ratio_top10_statistics['휴폐업구분']

X_stats_ksg_statistics = df_ttest_ksg_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_ksg_statistics = df_ttest_ksg_statistics['휴폐업구분']

X_ratio_stats_ksg_statistics = df_ratio_ksg_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_statistics = df_ratio_ksg_statistics['휴폐업구분']

X_ratio_and_top10_stats_ksg_statistics = df_ratio_top10_ksg_statistics.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_statistics = df_ratio_top10_ksg_statistics['휴폐업구분']

In [109]:
print('========================T-Test 기본 데이터+통계청===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_statistics, y_stats_statistics)
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+통계청=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_statistics, y_ratio_stats_statistics)
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+통계청====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_statistics, y_ratio_and_top10_stats_statistics)
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+통계청==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_statistics, y_stats_ksg_statistics)
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+통계청========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_statistics, y_ratio_stats_ksg_statistics)
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+통계청==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_statistics, y_ratio_and_top10_stats_ksg_statistics)
train_model(X_train, y_train, X_test, y_test)

========================T-Test 기본 데이터+통계청===========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 103), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 103), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9733, 정밀도: 0.8288, 재현율: 0.6101, F1: 0.7028
XGB모델 정확도: 0.9746, 정밀도: 0.8366, 재현율: 0.6313, F1: 0.7196
RandomForest모델 정확도: 0.9740, 정밀도: 0.8758, 재현율: 0.5796, F1: 0.6975
LogisticRegression모델 정확도: 0.9480, 정밀도: 0.1429, 재현율: 0.0013, F1: 0.0026
NaiveBayes모델 정확도: 0.9361, 정밀도: 0.0459, 재현율: 0.0119, F1: 0.0189
======================T-Test 기본+비율 데이터+통계청=========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 109), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 109), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9744, 정밀도: 0.8405, 재현율: 0.6220, F1: 0.7149
XGB모델 정확도: 0.9743, 정밀도: 0.8319, 재현율: 0.6300, F1: 0.7170
RandomForest모델 정확도: 0.9738, 정밀도: 0.8644, 재현율: 0.5836, F1: 0.6968
LogisticRegression모델 정확도: 0.9480, 정밀도: 0.1429, 재현율: 0.0013, F1: 0.0026
NaiveBayes모델 정확도: 0.9362, 정밀도: 0.0508, 재현율: 0.0133, F1: 0.0210
==================T-Test 

# 4. 재무비율 스코어

In [110]:
finance_ratio_score = pd.read_csv('./finance_ratio_score.csv', encoding='cp949')

In [111]:
finance_ratio_score = finance_ratio_score[['사업자등록번호', '결산년월', '재무비율점수']]

In [112]:
finance_ratio_score['사업자등록번호'] = finance_ratio_score['사업자등록번호'].astype(str)
finance_ratio_score['결산년월'] = finance_ratio_score['결산년월'].astype(str)

In [113]:
df_ttest_finance_score = pd.merge(df_ttest, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_finance_score = pd.merge(df_basic_ratio, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_finance_score = pd.merge(df_basic_ratio_and_top10, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_finance_score = pd.merge(df_basic_ttest_ksg, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_finance_score = pd.merge(df_ratio_ksg, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_finance_score = pd.merge(df_ratio_top10_ksg, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_statistics_finance_score = pd.merge(df_ttest_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_statistics_finance_score = pd.merge(df_ratio_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_statistics_finance_score = pd.merge(df_ratio_top10_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_statistics_finance_score = pd.merge(df_ttest_ksg_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_statistics_finance_score = pd.merge(df_ratio_ksg_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_statistics_finance_score = pd.merge(df_ratio_top10_ksg_statistics, finance_ratio_score, on=['사업자등록번호', '결산년월'], how='left')

In [114]:
# 일부 주소가 없는 기업들(검색해도 나오지 않는 기업들) 제거
df_ttest_finance_score.dropna(inplace=True)
df_ratio_finance_score.dropna(inplace=True)
df_ratio_top10_finance_score.dropna(inplace=True)

df_ttest_ksg_finance_score.dropna(inplace=True)
df_ratio_ksg_finance_score.dropna(inplace=True)
df_ratio_top10_ksg_finance_score.dropna(inplace=True)

In [119]:
X_stats_finance_score = df_ttest_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_finance_score = df_ttest_finance_score['휴폐업구분']

X_ratio_stats_finance_score = df_ratio_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_finance_score = df_ratio_finance_score['휴폐업구분']

X_ratio_and_top10_stats_finance_score = df_ratio_top10_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_finance_score = df_ratio_top10_finance_score['휴폐업구분']

X_stats_ksg_finance_score = df_ttest_ksg_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_ksg_finance_score = df_ttest_ksg_finance_score['휴폐업구분']

X_ratio_stats_ksg_finance_score = df_ratio_ksg_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_finance_score = df_ratio_ksg_finance_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_finance_score = df_ratio_top10_ksg_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_finance_score = df_ratio_top10_ksg_finance_score['휴폐업구분']

X_stats_statistics_finance_score = df_ttest_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_statistics_finance_score = df_ttest_statistics_finance_score['휴폐업구분']

X_ratio_stats_statistics_finance_score = df_ratio_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_statistics_finance_score = df_ratio_statistics_finance_score['휴폐업구분']

X_ratio_and_top10_stats_statistics_finance_score = df_ratio_top10_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_statistics_finance_score = df_ratio_top10_statistics_finance_score['휴폐업구분']

X_stats_ksg_statistics_finance_score = df_ttest_ksg_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_ksg_statistics_finance_score = df_ttest_ksg_statistics_finance_score['휴폐업구분']

X_ratio_stats_ksg_statistics_finance_score = df_ratio_ksg_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_statistics_finance_score = df_ratio_ksg_statistics_finance_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_statistics_finance_score = df_ratio_top10_ksg_statistics_finance_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_statistics_finance_score = df_ratio_top10_ksg_statistics_finance_score['휴폐업구분']

In [121]:
print('========================T-Test 기본 데이터+재무비율점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_finance_score , y_stats_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+재무비율점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_finance_score , y_ratio_stats_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+재무비율점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_finance_score , y_ratio_and_top10_stats_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+재무비율점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_finance_score , y_stats_ksg_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+재무비율점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_finance_score , y_ratio_stats_ksg_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+재무비율점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_finance_score , y_ratio_and_top10_stats_ksg_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('========================T-Test 기본 데이터+통계청+재무비율점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_statistics_finance_score , y_stats_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+통계청+재무비율점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_statistics_finance_score , y_ratio_stats_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+통계청+재무비율점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_statistics_finance_score , y_ratio_and_top10_stats_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+통계청+재무비율점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_statistics_finance_score , y_stats_ksg_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+통계청+재무비율점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_statistics_finance_score , y_ratio_stats_ksg_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+통계청+재무비율점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_statistics_finance_score , y_ratio_and_top10_stats_ksg_statistics_finance_score)
train_model(X_train, y_train, X_test, y_test)

========================T-Test 기본 데이터+재무비율점수===========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 96), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 96), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9725, 정밀도: 0.8146, 재현율: 0.6061, F1: 0.6951
XGB모델 정확도: 0.9734, 정밀도: 0.8222, 재현율: 0.6194, F1: 0.7065
RandomForest모델 정확도: 0.9739, 정밀도: 0.8693, 재현율: 0.5822, F1: 0.6974
LogisticRegression모델 정확도: 0.9481, 정밀도: 0.3846, 재현율: 0.0066, F1: 0.0130
NaiveBayes모델 정확도: 0.9358, 정밀도: 0.0493, 재현율: 0.0133, F1: 0.0209
======================T-Test 기본+비율 데이터+재무비율점수=========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 102), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 102), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9732, 정밀도: 0.8247, 재현율: 0.6114, F1: 0.7022
XGB모델 정확도: 0.9739, 정밀도: 0.8278, 재현율: 0.6247, F1: 0.7120
RandomForest모델 정확도: 0.9741, 정밀도: 0.8686, 재현율: 0.5875, F1: 0.7009
LogisticRegression모델 정확도: 0.9481, 정밀도: 0.3333, 재현율: 0.0053, F1: 0.0104
NaiveBayes모델 정확도: 0.9361, 정밀도: 0.0503, 재현율: 0.0133, F1: 0.0210
==================T-T

# 5. 통계청 스코어

In [122]:
statistics_score = pd.read_csv('./statistics_score.csv', encoding='cp949')

In [123]:
statistics_score = statistics_score[['사업자등록번호', '결산년월', '비재무점수']]

In [124]:
statistics_score['사업자등록번호'] = statistics_score['사업자등록번호'].astype(str)
statistics_score['결산년월'] = statistics_score['결산년월'].astype(str)

In [125]:
df_ttest_statistics_score = pd.merge(df_ttest, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_statistics_score = pd.merge(df_basic_ratio, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_statistics_score = pd.merge(df_basic_ratio_and_top10, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_statistics_score = pd.merge(df_basic_ttest_ksg, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_statistics_score = pd.merge(df_ratio_ksg, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_statistics_score = pd.merge(df_ratio_top10_ksg, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_statistics_statistics_score = pd.merge(df_ttest_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_statistics_statistics_score = pd.merge(df_ratio_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_statistics_statistics_score = pd.merge(df_ratio_top10_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_statistics_statistics_score = pd.merge(df_ttest_ksg_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_statistics_statistics_score = pd.merge(df_ratio_ksg_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_statistics_statistics_score = pd.merge(df_ratio_top10_ksg_statistics, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_finance_score_statistics_score = pd.merge(df_ttest_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_finance_score_statistics_score = pd.merge(df_ratio_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_finance_score_statistics_score = pd.merge(df_ratio_top10_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_finance_score_statistics_score = pd.merge(df_ttest_ksg_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_finance_score_statistics_score = pd.merge(df_ratio_ksg_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_finance_score_statistics_score = pd.merge(df_ratio_top10_ksg_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_statistics_finance_score_statistics_score = pd.merge(df_ttest_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_statistics_finance_score_statistics_score = pd.merge(df_ratio_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_statistics_finance_score_statistics_score = pd.merge(df_ratio_top10_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

df_ttest_ksg_statistics_finance_score_statistics_score = pd.merge(df_ttest_ksg_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_ksg_statistics_finance_score_statistics_score = pd.merge(df_ratio_ksg_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')
df_ratio_top10_ksg_statistics_finance_score_statistics_score = pd.merge(df_ratio_top10_ksg_statistics_finance_score, statistics_score, on=['사업자등록번호', '결산년월'], how='left')

In [126]:
# 일부 주소가 없는 기업들(검색해도 나오지 않는 기업들) 제거
df_ttest_statistics_score.dropna(inplace=True)
df_ratio_statistics_score.dropna(inplace=True)
df_ratio_top10_statistics_score.dropna(inplace=True)

df_ttest_ksg_statistics_score.dropna(inplace=True)
df_ratio_ksg_statistics_score.dropna(inplace=True)
df_ratio_top10_ksg_statistics_score.dropna(inplace=True)

df_ttest_finance_score_statistics_score.dropna(inplace=True)
df_ratio_finance_score_statistics_score.dropna(inplace=True)
df_ratio_top10_finance_score_statistics_score.dropna(inplace=True)

df_ttest_ksg_finance_score_statistics_score.dropna(inplace=True)
df_ratio_ksg_finance_score_statistics_score.dropna(inplace=True)
df_ratio_top10_ksg_finance_score_statistics_score.dropna(inplace=True)

In [127]:
X_stats_statistics_score = df_ttest_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_statistics_score = df_ttest_statistics_score['휴폐업구분']

X_ratio_stats_statistics_score = df_ratio_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_statistics_score = df_ratio_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_statistics_score = df_ratio_top10_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_statistics_score = df_ratio_top10_statistics_score['휴폐업구분']

X_stats_ksg_statistics_score = df_ttest_ksg_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_ksg_statistics_score = df_ttest_ksg_statistics_score['휴폐업구분']

X_ratio_stats_ksg_statistics_score = df_ratio_ksg_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_statistics_score = df_ratio_ksg_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_statistics_score = df_ratio_top10_ksg_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_statistics_score = df_ratio_top10_ksg_statistics_score['휴폐업구분']

X_stats_statistics_statistics_score = df_ttest_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_statistics_statistics_score = df_ttest_statistics_statistics_score['휴폐업구분']

X_ratio_stats_statistics_statistics_score = df_ratio_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_statistics_statistics_score = df_ratio_statistics_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_statistics_statistics_score = df_ratio_top10_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_statistics_statistics_score = df_ratio_top10_statistics_statistics_score['휴폐업구분']

X_stats_ksg_statistics_statistics_score = df_ttest_ksg_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_ksg_statistics_statistics_score = df_ttest_ksg_statistics_statistics_score['휴폐업구분']

X_ratio_stats_ksg_statistics_statistics_score = df_ratio_ksg_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_statistics_statistics_score = df_ratio_ksg_statistics_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_statistics_statistics_score = df_ratio_top10_ksg_statistics_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_statistics_statistics_score = df_ratio_top10_ksg_statistics_statistics_score['휴폐업구분']

X_stats_finance_score_statistics_score = df_ttest_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_finance_score_statistics_score = df_ttest_finance_score_statistics_score['휴폐업구분']

X_ratio_stats_finance_score_statistics_score = df_ratio_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_finance_score_statistics_score = df_ratio_finance_score_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_finance_score_statistics_score = df_ratio_top10_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_finance_score_statistics_score = df_ratio_top10_finance_score_statistics_score['휴폐업구분']

X_stats_ksg_finance_score_statistics_score = df_ttest_ksg_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_stats_ksg_finance_score_statistics_score = df_ttest_ksg_finance_score_statistics_score['휴폐업구분']

X_ratio_stats_ksg_finance_score_statistics_score = df_ratio_ksg_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_finance_score_statistics_score = df_ratio_ksg_finance_score_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_finance_score_statistics_score = df_ratio_top10_ksg_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_finance_score_statistics_score = df_ratio_top10_ksg_finance_score_statistics_score['휴폐업구분']

X_stats_statistics_finance_score_statistics_score = df_ttest_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_statistics_finance_score_statistics_score = df_ttest_statistics_finance_score_statistics_score['휴폐업구분']

X_ratio_stats_statistics_finance_score_statistics_score = df_ratio_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_statistics_finance_score_statistics_score = df_ratio_statistics_finance_score_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_statistics_finance_score_statistics_score = df_ratio_top10_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_statistics_finance_score_statistics_score = df_ratio_top10_statistics_finance_score_statistics_score['휴폐업구분']

X_stats_ksg_statistics_finance_score_statistics_score = df_ttest_ksg_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_stats_ksg_statistics_finance_score_statistics_score = df_ttest_ksg_statistics_finance_score_statistics_score['휴폐업구분']

X_ratio_stats_ksg_statistics_finance_score_statistics_score = df_ratio_ksg_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_stats_ksg_statistics_finance_score_statistics_score = df_ratio_ksg_statistics_finance_score_statistics_score['휴폐업구분']

X_ratio_and_top10_stats_ksg_statistics_finance_score_statistics_score = df_ratio_top10_ksg_statistics_finance_score_statistics_score.drop(['사업자등록번호', '결산년월', '업종대분류', '시도', '년도', '기업수', 
                            '폐업수', '폐업률', '취업자증감', '실업률', '특허', '가구별소득', '인구수', '전기사용량', '휴폐업구분'], axis=1)
y_ratio_and_top10_stats_ksg_statistics_finance_score_statistics_score = df_ratio_top10_ksg_statistics_finance_score_statistics_score['휴폐업구분']

In [128]:
print('========================T-Test 기본 데이터+비재무점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_statistics_score  , y_stats_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+비재무점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_statistics_score  , y_ratio_stats_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+비재무점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_statistics_score  , y_ratio_and_top10_stats_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+비재무점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_statistics_score  , y_stats_ksg_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+비재무점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_statistics_score  , y_ratio_stats_ksg_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+비재무점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_statistics_score  , y_ratio_and_top10_stats_ksg_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('========================T-Test 기본 데이터+지방지표+비재무점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_statistics_statistics_score  , y_stats_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+지방지표+비재무점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_statistics_statistics_score  , y_ratio_stats_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+지방지표+비재무점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_statistics_statistics_score  , y_ratio_and_top10_stats_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+지방지표+비재무점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_statistics_statistics_score  , y_stats_ksg_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+지방지표+비재무점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_statistics_statistics_score  , y_ratio_stats_ksg_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+지방지표+비재무점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_statistics_statistics_score  , y_ratio_and_top10_stats_ksg_statistics_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('========================T-Test 기본 데이터+재무비율점수+비재무점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_finance_score_statistics_score  , y_stats_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+재무비율점수+비재무점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_finance_score_statistics_score  , y_ratio_stats_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+재무비율점수+비재무점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_finance_score_statistics_score  , y_ratio_and_top10_stats_finance_score_statistics_score  )
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+재무비율점수+비재무점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_finance_score_statistics_score  , y_stats_ksg_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+재무비율점수+비재무점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_finance_score_statistics_score  , y_ratio_stats_ksg_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+재무비율점수+비재무점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_finance_score_statistics_score  , y_ratio_and_top10_stats_ksg_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('========================T-Test 기본 데이터+지방지표+재무비율점수+비재무점수===========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_statistics_finance_score_statistics_score  , y_stats_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('======================T-Test 기본+비율 데이터+지방지표+재무비율점수+비재무점수=========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_statistics_finance_score_statistics_score  , y_ratio_stats_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('==================T-Test 기본+비율+top10비율 데이터+지방지표+재무비율점수+비재무점수====================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_statistics_finance_score_statistics_score  , y_ratio_and_top10_stats_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=======================T-Test 기본 데이터+KSG+지방지표+재무비율점수+비재무점수==========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_stats_ksg_statistics_finance_score_statistics_score  , y_stats_ksg_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=====================T-Test 기본+비율 데이터+KSG+지방지표+재무비율점수+비재무점수========================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_stats_ksg_statistics_finance_score_statistics_score  , y_ratio_stats_ksg_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

print('=================T-Test 기본+비율+top10비율 데이터+KSG+지방지표+재무비율점수+비재무점수==================')
# 가설검정으로 선정한 데이터
X_train, y_train, X_test, y_test = preprocess_and_split_data(X_ratio_and_top10_stats_ksg_statistics_finance_score_statistics_score  , y_ratio_and_top10_stats_ksg_statistics_finance_score_statistics_score )
train_model(X_train, y_train, X_test, y_test)

========================T-Test 기본 데이터+비재무점수===========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 96), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 96), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9731, 정밀도: 0.8229, 재현율: 0.6101, F1: 0.7007
XGB모델 정확도: 0.9734, 정밀도: 0.8199, 재현율: 0.6220, F1: 0.7074
RandomForest모델 정확도: 0.9737, 정밀도: 0.8730, 재현율: 0.5743, F1: 0.6928
LogisticRegression모델 정확도: 0.9481, 정밀도: 0.3333, 재현율: 0.0053, F1: 0.0104
NaiveBayes모델 정확도: 0.9358, 정밀도: 0.0493, 재현율: 0.0133, F1: 0.0209
======================T-Test 기본+비율 데이터+비재무점수=========================
훈련 데이터셋의 입력 데이터의 크기: (58364, 102), 타겟값의 크기: (58364,)
테스트 데이터셋의 입력 데이터의 크기: (14592, 102), 타겟값의 크기: (14592,)
LGBM모델 정확도: 0.9735, 정밀도: 0.8309, 재현율: 0.6127, F1: 0.7053
XGB모델 정확도: 0.9748, 정밀도: 0.8386, 재현율: 0.6340, F1: 0.7221
RandomForest모델 정확도: 0.9738, 정밀도: 0.8735, 재현율: 0.5769, F1: 0.6949
LogisticRegression모델 정확도: 0.9481, 정밀도: 0.3333, 재현율: 0.0053, F1: 0.0104
NaiveBayes모델 정확도: 0.9361, 정밀도: 0.0503, 재현율: 0.0133, F1: 0.0210
==================T-Tes